Función que calcula el peso entre los bordes de los nodos

In [1]:
import math as m
import numpy as np
def weight(long1, lat1, long2, lat2, vm, h):
    mc = [1.0, 1.25, 1.5]
    d_lat = lat2 - lat1
    d_long = long2 - long1
    a = m.sin(d_lat/2)**2 + m.cos(lat1) * m.cos(lat2) * m.sin(d_long/2)**2
    c = 2 * m.atan(a)

    d = 6378 * c * 1000
    vm = vm / 3.6
    t = d/vm

    if (0 <= h <= 5) or (10 <= h <= 12) or (22 <= h <= 24):
        t = t * np.random.choice(mc, 1, p=[0.70,0.20,0.10])
    if 16 <= h <= 18:
        t = np.random.choice(mc, 1, p=[0.10,0.70,0.20])
    if (7 <= h <= 9) or (13 <= h <= 15) or (19 <= h <= 21):
        t = np.random.choice(mc, 1, p=[0.10,0.20,0.70])

    return round(float(t/60), 5)

Algoritmo de búsqueda de nodos Dijsktra

In [2]:
def dijsktra(graph, initial, end):
    shortest_paths = {initial: (None, 0)}
    current_node = initial
    visited = set()

    while current_node != end:
        visited.add(current_node)
        destinations = graph.edges[current_node]
        weight_to_current_node = shortest_paths[current_node][1]

        for next_node in destinations:
            weight = graph.weights[(current_node, next_node)] + weight_to_current_node
            if next_node not in shortest_paths:
                shortest_paths[next_node] = (current_node, weight)
            else:
                current_shortest_weight = shortest_paths[next_node][1]
                if current_shortest_weight > weight:
                    shortest_paths[next_node] = (current_node, weight)

        next_destinations = {node: shortest_paths[node] for node in shortest_paths if node not in visited}
        if not next_destinations:
            return "Route Not Possible"
        current_node = min(next_destinations, key=lambda k: next_destinations[k][1])

    path = []
    while current_node is not None:
        path.append(current_node)
        next_node = shortest_paths[current_node][0]
        current_node = next_node
    path = path[::-1]
    return path

Creación de la clase Graph y función para crear el grafo

In [3]:
from collections import defaultdict
class Graph():
    def __init__(self):
        self.edges = defaultdict(set)
        self.weights = {}

    def add_edge(self, s, e, w):
        self.edges[s].add(e)
        self.edges[e].add(s)
        self.weights[(s, e)] = w
        self.weights[(e, s)] = w

    def add_directed_edge(self, s, e, w):
        self.edges[s].add(e)
        self.weights[(s, e)] = w

def readFiles(file1, file2, file3, hour):
    nodes = []
    edges = []
    velocities = []
    with open(file1) as f:
        for line in f.readlines():
            nodes.append((float(line.split()[1]), float(line.split()[2])))

    with open(file2) as f:
        for line in f.readlines():
            edges.append((int(line.split()[1]), int(line.split()[2])))

    with open(file3) as f:
        for line in f.readlines():
            velocities.append(int(line))

    edges = sorted(edges, key=lambda x: x[0])
    G = Graph()

    for i in range(len(edges)):
        s = edges[i][0]
        e = edges[i][1]
        G.add_edge(s, e, weight(nodes[s][0], nodes[s][1], nodes[e][0], nodes[e][1], velocities[i], hour))
    return G

Función que encapsula todo el programa

In [16]:
def App():
    h = int(input("Ingrese la hora en la que se calculará la ruta para calcular el tráfico aproximado de la zona: "))
    s = int(input("Ingrese el nodo de salida: "))
    e = int(input("Ingrese el nodo de llegada: "))
    G = readFiles("nodes2.txt", "edges2.txt", "max_vel.txt", h)
    print("El camino más corto del punto {} al punto {} es el siguiente: {}".format(s, e, dijsktra(G, s, e)))

App()

El camino más corto del punto 0 al punto 5 es el siguiente: [0, 5]
